# 가중치 조건
1. ADR과의 거리: 기간동안 ADR의 전체 평균을 0으로 두고 스케일을 정규화 한다. ADR의 상하한은  ADR과 값차이에 가중치의 가중치를 곱해 최종 가중치를 부여한다. 
2. 정배열과 역배열의 정도: 
3. 볼린저밴드 상의 위치:55
# 하나씩 적용(3종류)+두개씩 적용(3종류)+세개 한번에 적용(1종류) 총 7 경우의 수
# 각 경우 가중치의 가중치 최적화

# adr에 대한 민감도를 arima로 예측
# adr과 등락률의 상관관계 구하기 --> 음의 상관관계이든 양의 상관관계이든 상관관계에 있는 종목이 상단과 하단을 차지 
# adr예측 --> 상관관계에따른 순위매기기
# 샀을 때 점수 팔았을 때 점수

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import warnings
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정


train = pd.read_csv('./train.csv')
kosdaq_kospi = pd.read_csv('./stock_data/kosdaq_kospi_list.csv', encoding='euc-kr')
kospi = kosdaq_kospi["코스피"]
kosdaq = list(kosdaq_kospi["코스닥"])
adr_final = pd.read_csv('./stock_data/adr_final.csv', encoding='euc-kr') # 코스피 종목수
adr_final["Date"] = adr_final["Date"].str[1:]
adr_final["kospi_pm"]=adr_final["kospi_adr"]-adr_final["kospi_adr"].shift(1)
adr_final["kosdaq_pm"]=adr_final["kosdaq_adr"]-adr_final["kosdaq_adr"].shift(1)
 
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame()

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

## 각 종목코드에 대해서 모델 학습 및 추론 반복
tmp = []
for code in tqdm(unique_codes):
#code = "A103840"

    # 20일선 가격의 상승분과 비교
    train_close = train[train['종목코드'] == code][['일자', '종가', '종목코드']]
    #train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index(np.arange(len(train_close)), inplace=True)
    train_close.columns = ['Date', '종가','종목코드']
    train_close["shift"] = train_close["종가"].shift(1)
    train_close["증감"] = ((train_close["종가"] - train_close["shift"]) / train_close["shift"])  # 당일종가-전일종가/전일종가 로 등락폭 계산
    train_close["Date"] = train_close['Date'].astype(int)
    adr_final["Date"]=adr_final["Date"].astype(int)

    train_adr = pd.merge(left = train_close, right = adr_final, how = "inner", on = "Date")

    if code in kosdaq:
       
        X = train_adr["kosdaq_pm"].values
        X = X.reshape(-1, 1)
        Y = train_adr["증감"].values
        Y = Y.reshape(-1, 1)
        lr.fit(X, Y)
        #r값이 몇 이하면 계산제외

        tc = train_adr["kosdaq_adr"]
        ### 모델 선언, 학습 및 추론
        model = ARIMA(tc, order=(2, 1, 2))
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측


        for i in predictions:
            
            final_return += (i*lr.coef_[0]+lr.intercept_) / predictions.iloc[0]

        # 결과 저장
        results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


    else:
        X = train_adr["증감"].values
        X = X.reshape(-1, 1)
        Y = train_adr["kospi_pm"].values
        Y = Y.reshape(-1, 1)
        lr.fit(X, Y)
        
        tc = train_adr["kospi_adr"]
        ### 모델 선언, 학습 및 추론
        model = ARIMA(tc, order=(2, 1, 2))
        model_fit = model.fit()
        predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측


        # 최종 수익률 계산
        final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

        # 결과 저장
        results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)
    tmp.append(lr.coef_[0])


    # adr과 거리가 멀어야 상위권, 비슷하면 하위권(절대값 씌운경우)

    tc = train_adr["가중치"]
    ### 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df = results_df.sort_values(by='순위')
results_df.to_csv('./stock_data/result_df.csv', encoding='euc-kr')
results_df

In [ ]:
import pandas as pd
import numpy as np
import random
import os


from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pd.read_csv('./train.csv')
kosdaq_kospi = pd.read_csv('./stock_data/kosdaq_kospi_list.csv', encoding='euc-kr')
kospi = kosdaq_kospi["코스피"]
kosdaq = list(kosdaq_kospi["코스닥"])
adr_final = pd.read_csv('./stock_data/adr_final.csv', encoding='euc-kr') # 코스피 종목수
adr_final["Date"] = adr_final["Date"].str[1:]


adr_final["nom_kospi_adr"] = adr_final["kospi_adr"].apply(lambda x: (x-100)/1000)
adr_final["nom_kosdaq_adr"] = adr_final["kosdaq_adr"].apply(lambda x: (x-100)/1000)

print(adr_final["kospi_adr"].max())     # 153.6
print(adr_final["kospi_adr"].min())     # 52.11
print(adr_final["kosdaq_adr"].max())    # 156.34
print(adr_final["kosdaq_adr"].min())    # 51.77


# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame()

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

## 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
#code = "A103840"

    # 20일선 가격의 상승분과 비교
    train_close = train[train['종목코드'] == code][['일자', '종가', '종목코드']]
    #train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index(np.arange(len(train_close)), inplace=True)
    train_close.columns = ['Date', '종가','종목코드']
    train_close["5평균"] = train_close['종가'].rolling(window=1).mean()
    train_close["shift"] = train_close["5평균"].shift(1)
    train_close["5일선증감"] = ((train_close["5평균"] - train_close["shift"]) / train_close["shift"])  # 당일종가-전일종가/전일종가 로 등락폭 계산

    train_close["Date"] = train_close['Date'].astype(int)
    adr_final["Date"]=adr_final["Date"].astype(int)
    train_adr = pd.merge(left = train_close, right = adr_final, how = "inner", on = "Date")
    if code in kosdaq:
        train_adr["가중치"] = -train_adr["5일선증감"]+train_adr["nom_kosdaq_adr"]
    else:
        train_adr["가중치"] = -train_adr["5일선증감"]+train_adr["nom_kospi_adr"]

    # adr과 거리가 멀어야 상위권, 비슷하면 하위권(절대값 씌운경우)

    tc = train_adr["가중치"]
    ### 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df = results_df.sort_values(by='순위')
results_df.to_csv('./stock_data/result_df.csv', encoding='euc-kr')
results_df


In [16]:
sample_submission = pd.read_csv('./sample_submission.csv')
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission.to_csv('test_submission.csv', index=False)


In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df = results_df.sort_values(by='순위')
results_df

# from pykrx import stock
# import pandas as pd

# buy = results_df.head(200)
# sell = results_df.tail(200)
# result = pd.concat([buy,sell])
# result.reset_index(inplace = True, drop = True)

# tmp_first_close = []
# tmp_final_close = []
# for i in result["종목코드"]:
#     data = stock.get_market_ohlcv_by_date("20230531","20230621", i[1:])
#     tmp_first_close.append(data.iloc[0]['종가'])
#     tmp_final_close.append(data.iloc[-1]['종가'])
# result["첫날종가"]=pd.Series(tmp_first_close)
# result["마지막날종가"]=pd.Series(tmp_final_close)


# result.to_csv('./stock_data/result_day1.csv', encoding='euc-kr')

# result


In [5]:
#results_df.to_csv('./stock_data/results_df.csv', encoding='euc-kr')

sample_submission = pd.read_csv('./sample_submission.csv')
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission.to_csv('test_submission.csv', index=False)


In [66]:
import requests
from bs4 import BeautifulSoup
import xmltodict
import json
import requests
from bs4 import BeautifulSoup
from pykrx import stock
import pandas as pd

data = stock.get_market_ohlcv_by_date("20210608","20210608", "005930")



results_tr = results_df.transpose()
results_tr.reset_index(inplace=True)
pred_date = [ "code","20230531", "20230601", "20230602", "20230605", "20230607", "20230608","20230609","20230612", "20230613", "20230614", "20230615", "20230616", "20230619", "20230620", "20230621"]
results_tr.columns = pred_date
prediction = pd.DataFrame()
for i in pred_date[1:]:
    result = results_tr[["code", i]]
    result = result.sort_values(by=i, ascending=False)
    result.reset_index(inplace=True)
    result = result.drop("index", axis=1)
    prediction[i]=result["code"]
    
buy = prediction.head(200)
sell = prediction.tail(200)
result_buy = pd.DataFrame(columns=pred_date[1:])
result_sell = pd.DataFrame(columns=pred_date[1:])

# sell.iloc[0][pred_date[1]]

for i in pred_date[1:]:
    print(i)
    tmp=[]
    for j in buy[i]:
        data = stock.get_market_ohlcv_by_date(i,i,j[1:])
        tmp.append(((data.iloc[0]['종가']-data.iloc[0]['시가'])/data.iloc[0]['시가'])*100)
    result_buy[i]=tmp
    tmp=[]
    for j in sell[i]:
        data = stock.get_market_ohlcv_by_date(i,i,j[1:])
        tmp.append(((data.iloc[0]['시가']-data.iloc[0]['종가'])/data.iloc[0]['종가'])*100)
    result_sell[i]=tmp

    
result_buy.to_csv('./stock_data/result_buy.csv', encoding='euc-kr')
result_sell.to_csv('./stock_data/result_sell.csv', encoding='euc-kr')





20230531
20230601
20230602
20230605
20230607
20230608
20230609
20230612
20230613
20230614
20230615
20230616
20230619
20230620
20230621


In [64]:
from pykrx import stock
import pandas as pd
stock_code = pd.read_csv("./sample_submission.csv")
stock_code['종목코드'] = stock_code['종목코드'].str[1:]    
data = stock.get_market_ohlcv_by_date("20210608","20210608", "005930")
data.iloc[0]['종가']
    



81900.0

In [58]:
import pandas as pd
stock_code = pd.read_csv("./sample_submission.csv")
stock_code['종목코드'] = stock_code['종목코드'].str[1:]

import requests
from bs4 import BeautifulSoup
import xmltodict
import json

 
import requests
from bs4 import BeautifulSoup
 
# 네이버 금융에서 종목 가격정보와 거래량을 가져오는 함수: get_price

def get_price(company_code):
    # count=3000에서 3000은 과거 3,000 영업일간의 데이터를 의미. 사용자가 조절 가능
    url = "https://fchart.stock.naver.com/sise.nhn?symbol={}&timeframe=day&count=34&requestType=0".format(company_code)
    get_result = requests.get(url)
    bs_obj = BeautifulSoup(get_result.content, "html.parser")
    
    # information
    inf = bs_obj.select('item')
    columns = ['Date', 'Open' ,'High', 'Low', 'Close', 'Volume']
    df_inf = pd.DataFrame([], columns = columns, index = range(len(inf)))
    df_inf['code'] = company_code

    for i in range(len(inf)):
        df_inf.iloc[i] = str(inf[i]['data']).split('|')
    return df_inf

tmp = []
for row in stock_code['종목코드']:
    tmp.append(get_price(row))
df_final = pd.concat(tmp, ignore_index=True)
df_final.to_csv('./stock_data/stockprice.csv', encoding='euc-kr')



ValueError: Must have equal len keys and value when setting with an iterable

In [65]:
import pandas as pd
stock_code = pd.read_csv("./sample_submission.csv")
stock_code['종목코드'] = stock_code['종목코드'].str[1:]
stock_code

,종목코드,순위
0,000020,1
1,000040,2
2,000050,3
3,000070,4
4,000080,5
...,...,...
1995,375500,1996
1996,378850,1997
1997,383220,1998
1998,383310,1999


In [ ]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

## 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
#code = "A103840"
# # 학습 데이터 생성 : 등락에 가중치 부여
    train_vol = train[train['종목코드'] == code][['일자', '고가', '저가', '거래량']] # 거래량을 통하여 거래대금 계산 # 각 종목의 모든 날짜 데이터를 가져온다.
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    train_vol['일자'] = pd.to_datetime(train_vol['일자'], format='%Y%m%d')
    train_vol.set_index('일자', inplace=True)

    df_adr = train_close['종가'].copy() # 볼린저밴드 밴드폭 가중치를 주기 위한 카피본

    # 등락폭 계산
    train_close["shift"] = train_close["종가"].shift(1)
    train_close["가중치"] = ((train_close["종가"] - train_close["shift"]) / train_close["shift"])  # 당일종가-전일종가/전일종가 로 등락폭 계산
    train_close.drop("shift", axis=1)       #.to_clipboard(index=False)

    # 날짜 비교, 코스닥코스피 분류, 등락폭-당일 adr
    train_close[train_close['가중치'] < 0.285] = 0.05


    # 거래대금 250억에서 800억 사이에 종목 가중치 부여
    tp[(0.25 > tp) | (tp > 0.8)] = 0.0

    train_close['가중치'] += tp


    # 이격도 차이 배열 정리
    for i in range(len(df_b)+1):
        if i > 0 and i+59 < len(df_b):
            dis[-i-1] = disparity(df_b[-(60+i) : -i ])
        elif i == 0 :
            dis[-1] = disparity(df_b[-60 : ])
        else :
            dis[-i] = 0



    ## 이격도, 볼밴 상하한, 밴드폭을 이용하여 가중치 부여
    for i in range(len(df_b)+1):
        if i > 0 and i+19 < len(df_b):
            upb, downb, bw = make_band(df_b[ -(20+i) : -i ])

            if 0.15 <= bw and 0.85 >= bw and df_b[-i-1] > upb :
                train_close['가중치'].iloc[-i-1] += bw
            elif 0.15 <= bw and 0.85 >= bw and df_b[-i-1] < downb :
                train_close['가중치'].iloc[-i-1] -= bw



        elif i == 0 :
            upb, downb, bw = make_band(df_b[ -20 :  ])

            if 0.15 <= bw and 0.85 >= bw and df_b[-1] > upb :
                train_close['가중치'].iloc[-1] += bw
            elif 0.15 <= bw and 0.85 >= bw and df_b[-1] < downb :
                train_close['가중치'].iloc[-1] -= bw

        if i > 0 and i+59 < len(df_b):
            if dis[-i-1] >= dis[-i-2] + 10 :
                train_close['가중치'].iloc[-i-1] += (dis[-i-1] - dis[-i-2])/100
            elif dis[-i-1] <= dis[-i-2] - 7 :
                train_close['가중치'].iloc[-i-1] -= (dis[-i-2] - dis[-i-1])/100

        elif i == 0 :
            if dis[-1] >= dis[-2] + 10 :
                train_close['가중치'].iloc[-1] += (dis[-1] - dis[-2])/100
            elif dis[-1] <= dis[-2] - 7 :
                train_close['가중치'].iloc[-1] -= (dis[-2] - dis[-1])/100

    tc = train_close['가중치']  # 첫 번째 루프에 대한 데이터 약 400일정도



    ### 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)